WARNING: this example is not yet fully functional
https://github.com/imperial-qore/line-solver/wiki/Getting-started#example-6-a-queueing-network-with-caching

In [1]:
from line_solver import *
GlobalConstants.setVerbose(VerboseLevel.STD)

In [2]:
model = Network('Model')
# Block 1: nodes
clientDelay = Delay(model, 'Client')
cacheNode = Cache(model, 'Cache', 1000, 50, ReplacementStrategy.LRU)
cacheDelay = Delay(model, 'CacheDelay')

In [3]:
# Block 2: classes
clientClass = ClosedClass(model, 'ClientClass', 1, clientDelay, 0)
hitClass = ClosedClass(model, 'HitClass', 0, clientDelay, 0)
missClass = ClosedClass(model, 'MissClass', 0, clientDelay, 0)

clientDelay.setService(clientClass, Immediate())
cacheDelay.setService(hitClass, Exp.fitMean(0.2))
cacheDelay.setService(missClass, Exp.fitMean(1.0))

cacheNode.setRead(clientClass, Zipf(1.4, 1000))
cacheNode.setHitClass(clientClass, hitClass)
cacheNode.setMissClass(clientClass, missClass)

In [4]:
# Block 3: topology
P = model.initRoutingMatrix()
# routing from client to cache
P.set(clientClass, clientClass, clientDelay, cacheNode, 1.0)
# routing out of the cache
P.set(hitClass, hitClass, cacheNode, cacheDelay, 1.0)
P.set(missClass, missClass, cacheNode, cacheDelay, 1.0)
# return to the client
P.set(hitClass, clientClass, cacheDelay, clientDelay, 1.0)
P.set(missClass, clientClass, cacheDelay, clientDelay, 1.0)
# routing from cacheNode
model.link(P)

In [5]:
# SSA not working on this case in Java but MVA works fine
# Block 4: solution
mvaAvgTable = SolverMVA(model,'verbose',True).getAvgTable()

      Station     JobClass      QLen          Util     RespT    ResidT  ArvR  \
0      Client  ClientClass  0.000015  2.610757e-08  0.000006  0.000006   0.0   
4  CacheDelay     HitClass  0.402683  4.026893e-01  0.199997  0.154240   0.0   
5  CacheDelay    MissClass  0.597302  5.973106e-01  0.999985  0.228785   0.0   

       Tput  
0  2.610757  
4  2.013447  
5  0.597311  


In [6]:
# ssaAvgTable = SolverSSA(model,'samples',20000,'seed',1,'verbose',True).getAvgTable()

In [7]:
# ssaAvgTablePara = SolverSSA(model,'samples',20000,'seed',1,'verbose',True,'parallel').getAvgTable()